# Quick performance comparison of galibrate and gaft

In this notebook we'll do a rough performance comparison of [galibrate](https://github.com/blakeaw/GAlibrate) (Numba implementation) and [gaft](https://github.com/PytLab/gaft) (serial version). 

For this test, we're fitting three data points with a linear equation by minimizing the chi^2 value; this was adapted from the [Gleipnir 3point_line example](https://github.com/LoLab-VU/Gleipnir/tree/master/examples/3point_line). This model has two parameters: m and b (from y=mx+b). 

To run this notebook, you will need to install
  * galibrate
  * gaft
  * numba

Here is the data we are fitting against:

In [2]:
# Setup the data points that are being fitted.
data_x = np.array([1., 2., 3.])
data_y = np.array([1.4, 1.7, 4.1])
data_yerr = np.array([0.2, 0.15, 0.2])

## galibrate set up and run

In [1]:
import numpy as np
from galibrate.sampled_parameter import SampledParameter
from galibrate import GAO

/home/blake/src/python/GAlibrate/galibrate/gao.py:10: RuntimeWarning: ------Running GAO with numba optimization.------
  warnings.warn("------Running GAO with numba optimization.------", RuntimeWarning)


In [3]:
# Define the fitness function
def fitness(chromosome):
    y = chromosome[1] * data_x + chromosome[0]
    chisq = np.sum(((data_y - y) / data_yerr)**2)
    if np.isnan(chisq):
        return -np.inf
    return -chisq / 2.

In [4]:
# Set up the list of sampled parameters: the range is (-5:5) --
parm_names = list(['m', 'b'])
sampled_parameters = [SampledParameter(name=p, loc=-5.0, width=10.0) for p in parm_names]

In [5]:
# Set the active point population size
population_size = 5000
generations = 100
mutation_rate = 0.1

In [6]:
gao = GAO(sampled_parameters,
         fitness,
         population_size,
         generations = generations,
         mutation_rate = mutation_rate)

In [7]:
ti_gal = %timeit -r 2 -n 3 -q -o gao.run()

In [8]:
print("Average run time (seconds): {}".format(ti_gal.average))

Average run time (seconds): 3.895735375665633


In [9]:
print(gao._fittest_chromosome, gao._fittest_fitness)

[-0.43165853  1.35146893] -12.98183701362593


## gaft set up and run

In [10]:
from gaft.components import BinaryIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft import GAEngine

In [11]:
indv = BinaryIndividual(ranges=[(-5., 5.), (-5.,5.)], eps=0.001)
population = Population(indv_template=indv, size=population_size).init()
selection = TournamentSelection()
crossover = UniformCrossover(pc=0.8, pe=0.5)
mutation = FlipBitMutation(pm=mutation_rate)
engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation, analysis=None)

In [12]:
# Define the fitness function
@engine.fitness_register
def gaft_fitness(indv):
    print(indv.solution)
    chromosome = indv.solution
    print(chromosome)
    y = chromosome[1] * data_x + chromosome[0]
    chisq = np.sum(((data_y - y) / data_yerr)**2)
    if np.isnan(chisq):
        #return -np.inf
        return -1000000.0
    return float(-chisq / 2.)

In [13]:
%%capture
ti_gaf = %timeit -n 3 -r 2 -o -q engine.run(ng=generations)

In [14]:
print("Average run time (seconds): {}".format(ti_gaf.average))

Average run time (seconds): 124.08712170616732


In [15]:
best_indv = engine.population.best_indv(engine.fitness)

In [16]:
print(best_indv.solution, engine.fitness(best_indv))

[-0.4443359375, 1.35009765625]
[-0.4443359375, 1.35009765625]
[-0.4443359375, 1.35009765625] -12.970588498645355


## Relative performance

In [17]:
speedup_factor = ti_gaf.average/ti_gal.average

In [18]:
print("Speedup factor galibrate/gaft: {}".format(speedup_factor))

Speedup factor galibrate/gaft: 31.852040690768312
